# Hands-On Exercise 11.2: Factories

## Objective

In this exercise, you will use a Factory to allow centralized decision making and control over the exact object type created. It also creates opportunities to provide dependency injection.

The starting point continues from the bonus of the previous exercise. The bonus modifications were made to the user interface to determine which shuffler to use and then construct the appropriate shuffler object. This approach makes the user interface and the card library rigid. Adding shufflers requires that we make changes in both places, and we've been told there will be numerous additional shufflers to be added. Furthermore, if we ever want to use the card library for another game, it would be necessary to duplicate the shuffler creation. We need to break this dependency, a job for a Factory.

#### Examine the existing code in the application.

Start Visual Studio and open the solution `C:\Course\510D\Exercises\Ex112\Blackjack.sln`.

In the `Blackjack` project, right-click `SelectShufflerDialog` and pick **View Code**.

Notice it is hard-coded to know all the shufflers and how to construct them.

#### Add the factory class.

Right-click the `CardLib` project and add a new class called `ShufflerFactory`.

<font color="red">**Make sure you put the factory in the `CardLib` class, not in the `Blackjack` user interface project.**</font>

Change the declaration of class `ShufflerFactory` to be `public static`.

Declare a `public static` method named `Create`. It should accept a `string` parameter called `shufflerText` and return an `IShuffler`. At this point, have it return `null` until we have some test logic.

#### Add the shuffler factory tests.

Right-click the `BlackjackTests` project and select **Add | New Item...**. In the templates, click **Test** and choose **Basic Unit Test**. Name it `ShufflerFactoryTests`.

Add using `CardLib;` at the top.

Rename `TestMethod1` to be `ShufflerCreationTest`.

In this method use `IShuffler shuffler = ShufflerFactory.Create("Knuth Shuffler");`.

In the next line add `Assert.IsNotNull(shuffler);`.

Assert that the object created matches the one expected. Use `Assert.AreEqual("KnuthShuffler", shuffler.GetType().Name);`.

Repeat the previous three steps, except check for the Modified Knuth Shuffler.

Add a test to confirm that the factory will return `null` for an unrecognized shuffler.

Your test should look something like:

```C#
[TestMethod]
public void ShufflerCreationTest()
{
    IShuffler shuffler = ShufflerFactory.Create("Knuth Shuffler");
    Assert.IsNotNull(shuffler); Assert.AreEqual("KnuthShuffler", shuffler.GetType().Name);
    
    shuffler = ShufflerFactory.Create("Modified Knuth Shuffler");
    Assert.IsNotNull(shuffler); Assert.AreEqual("ModifiedKnuthShuffler", shuffler.GetType().Name);
    
    shuffler = ShufflerFactory.Create("No Such Shuffler"); 
    Assert.IsNull(shuffler);
}
```

Run all tests. The `ShufflerCreationTest` should fail.

#### Complete the implementation of the `ShufflerFactory.Create(...)` method.

Add conditional logic to check if the `shufflerText` is `"Knuth Shuffler"` and if so, return a new `KnuthShuffler`.

Repeat for the modified Knuth Shuffler.
    
Your C# code might look something like this:

```C#
public static IShuffler Create(string shufflerText)
{
    if (shufflerText == "Knuth Shuffler") 
        return new KnuthShuffler();

    if (shufflerText == "Modified Knuth Shuffler") 
        return new ModifiedKnuthShuffler();

    return null;
}
```

Run all tests. Repeat fixing problems (if necessary) until you have green lights.

#### Modify the `SelectShufflerDialog` to use the factory.

Open `SelectShufflerDialog` in code view.

In the `btnSelect_Click(...)` method, delete all the code except for the dispose statement.

Insert the code to make a shuffler instance using the factory. Use `IShuffler shuffler = ShufflerFactory.Create(ddlSelectShuffler.Text);`.

In the next line, add a conditional to ensure the shuffler is not `null`. If it is not `null`, then call `deck.ChangeShuffler(shuffler)`.

Your C# code might look something like this:

```C#
private void btnSelect_Click(object sender, EventArgs e)
{
    IShuffler shuffler = ShufflerFactory.Create(ddlSelectShuffler.Text);
    if (shuffler != null) 
        deck.ChangeShuffler(shuffler);
    this.Dispose();
}
```

Run all tests for green lights, then run the game and confirm you can select different shufflers.

This is better, but we are only part way there. Although the `SelectShufflerDialog` no longer must determine which shuffler to construct, it still has to know what they are. Time to inject the dependency to the `ShufflerFactory`.

#### Modify the ShufflerFactory to provide a list of all the shuffler choices available.

Open `ShufflerFactory.cs` for editing and add a public property called `Choices` that returns an array of strings. It should have a private set and be initialized with the text descriptions of the shufflers.

You can use:

```C#
public static string[] Choices { get; private set; } = { "Knuth Shuffler", "Modified Knuth Shuffler" };
```

In the `Create` method, replace the explicit strings with `Choices[0]`, `Choices[1]`, etc.

Making this change will mean the only place the description strings are defined is in the `Choices` array. They can be changed to anything and the Factory and everything else will still work properly.

Your `ShufflerFactory` class should now look like:

```C#
public abstract class ShufflerFactory
{
    public static string[] Choices { get; private set; } = { "Knuth Shuffler", "Modified Knuth Shuffler" };

    public static IShuffler Create(string shufflerText)
    {
        if (shufflerText == Choices[0])
            return new KnuthShuffler();
        if (shufflerText == Choices[1]) 
            return new ModifiedKnuthShuffler();
        return null;
    }
}
```

Run all tests to ensure you still have green lights.

#### Edit the `SelectShufflerDialog` to inject the remaining dependencies to the `ShufflerFactory`.

Back in code view for the `SelectShufflerDialog`, move to the `Form_Load(...)` event method. Replace the existing code by:

- Looping through each of the choices provided by the `ShufflerFactory.Choices` array
- Adding each to the `ddlSelectShuffler.Items` collection
 
You can use:

```C#
private void Form1_Load(object sender, EventArgs e)
{
    foreach (string choice in ShufflerFactory.Choices)
    {
        ddlSelectShuffler.Items.Add(choice);
    }
}
```

Notice that there is no longer any knowledge in the dialog about what shufflers are available or how to construct them.

Run all tests for green lights, then run the program and confirm that different shufflers can still be selected.

When adding shufflers in the future, it will not be necessary to touch the user interface to support it. All dependencies have been injected into the `ShufflerFactory`. All that will be needed is to add a new shuffler class, and then edit the `ShufflerFactory` to make it available.

Change the text of the `"Knuth Shuffler"` in the `Choices` array to be `"Simple Knuth Shuffler"`.

Modify the shuffler test and change `"Knuth Shuffler"` to `"Simple Knuth Shuffler"`. This is still explicit.

Run and test the selection choices.

We needed to change the text in only one place. Notice that the selections and creation all still work correctly.

Now would be a good time to:

- Do a **Build | Rebuild Solution**
- Run all tests to get green lights
- Execute the program to visually check if it is still working properly

## Congratulations! You have implemented the Factory pattern in this application. Carry on to the bonus if you have more time.

# Bonus (Optional)

Add an additional shuffler.

#### Add an additional test to ensure all choices can be created.

Open `ShufflerFactoryTests.cs` for editing and add a new test method called `ShufflerAllChoicesTest()`.

In this method, add the logic to loop through all `ShufflerFactory.Choices` and call `ShufflerFactory.Create(...)` to create an instance for each choice. Inside the loop, confirm that the instance created is not `null`.

Your test should look something like:

```C#
[TestMethod]
public void ShufflerAllChoicesTest()
{
    foreach (string choice in ShufflerFactory.Choices)
    {
        IShuffler shuffler = ShufflerFactory.Create(choice);
        Assert.IsNotNull(shuffler);
    }
}
```

This test will confirm that all shufflers including new ones can be created.

Run all tests to get green lights.

#### Add an additional shuffler; one has been pre-written for you.

Right-click the `CardLib` project and select **Add | Existing Item...**. Navigate to and select `C:\Course\510D\Code\TwoEndedShuffler.cs`.

Get a clean compile.

Modify the `ShufflerFactory` to include the `TwoEndedShuffler`.

Your shuffler factory should now look like:

```C#
public abstract class ShufflerFactory
{
    public static string[] Choices { get; private set; } = { "Simple Knuth Shuffler", "Modified Knuth Shuffler", "Two Ended Shuffler" };

    public static IShuffler Create(string shufflerText)
    {
        if (shufflerText == Choices[0])
            return new KnuthShuffler();
        if (shufflerText == Choices[1]) 
            return new ModifiedKnuthShuffler();
        if (shufflerText == Choices[2]) 
            return new TwoEndedShuffler();
        return null;
    }
}
```

Run all tests to get green lights. This will confirm that our new shuffler is being properly created.

Run the application and go to shuffler selection and select Two Ended Shuffler.

Note that the Two Ended Shuffler has been added to the view even though we did not make any modifications to the view to add this support.

Now would be a good time to:

- Do a **Build | Rebuild Solution**
- Run all tests to get green lights
- Execute the program to visually check if it is still working properly

## Congratulations! You have completed the bonus steps.